In [5]:
# -*- coding: utf-8 -*-
"""
Transformer Embedding Ensemble with Weighted Soft Voting
Models: RoBERTa, BERT, BART, MiniLM, DistilBERT, DeBERTa
Classifiers: RandomForest, GaussianNB, XGBoost, Linear SVM
Evaluation: 10-fold Cross Validation
"""

import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

# ---------------------------
# Step 1: Load Data
# ---------------------------
print("Step 1: Loading and preparing data...")

try:
    train_df1 = pd.read_csv(r"C:\Users\hp\OneDrive\Desktop\IMP FILES\FINAL PROJECT\Causality\dataset 1\train_subtask1.csv")
    train_df2 = pd.read_csv(r"C:\Users\hp\OneDrive\Desktop\IMP FILES\FINAL PROJECT\Causality\dataset 1\dev_subtask1.csv")
    test_df = pd.read_csv(r"C:\Users\hp\OneDrive\Desktop\IMP FILES\FINAL PROJECT\Causality\dataset 1\test_subtask1_text.csv")
except FileNotFoundError as e:
    print(f"Error: {e}. Please check dataset paths.")
    exit()

train_df = pd.concat([train_df1, train_df2], ignore_index=True)
X_train_raw = train_df["text"]
y_train = train_df["label"]
X_test_raw = test_df["text"]

# ---------------------------
# Step 2: Preprocessing
# ---------------------------
print("Step 2: Preprocessing text...")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

X_train_processed = X_train_raw.apply(preprocess_text)
X_test_processed = X_test_raw.apply(preprocess_text)

# ---------------------------
# Step 3: Transformer Embeddings
# ---------------------------
print("Step 3: Generating embeddings...")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transformer_models = {
    "roberta": "roberta-base",
    "bert": "bert-base-uncased",
    "bart": "facebook/bart-base",
    "minilm": "nreimers/MiniLM-L6-H384-uncased",
    "distilbert": "distilbert-base-uncased",
    "deberta": "microsoft/deberta-base"
}

def get_embeddings(texts, model_name, batch_size=16, max_len=128):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    all_embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc=f"Embedding {model_name}"):
            batch = texts[i:i+batch_size].tolist()
            encodings = tokenizer(batch, padding=True, truncation=True,
                                  max_length=max_len, return_tensors="pt").to(device)
            outputs = model(**encodings)
            cls_embeddings = outputs.last_hidden_state[:,0,:]  # [CLS] token
            all_embeddings.append(cls_embeddings.cpu().numpy())
    return np.vstack(all_embeddings)

# Extract embeddings for each model
embeddings = {}
for name, model_name in transformer_models.items():
    embeddings[name] = get_embeddings(X_train_processed, model_name)

# Concatenate embeddings into one feature vector
X_train_emb = np.hstack(list(embeddings.values()))

Step 1: Loading and preparing data...
Step 2: Preprocessing text...
Step 3: Generating embeddings...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Embedding microsoft/deberta-base: 100%|██████████████████████████████████████████████| 203/203 [02:14<00:00,  1.51it/s]


In [2]:
# -*- coding: utf-8 -*-
"""
Transformer Embeddings + SVM-RBF Classifier
10-Fold Cross Validation with Best Hyperparameters
"""

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

# ---------------------------
# Config
# ---------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
FOLDS = 10
RANDOM_STATE = 42

np.random.seed(SEED)
torch.manual_seed(SEED)

# ---------------------------
# Embedding extraction
# ---------------------------
def get_embeddings(model_name, texts, batch_size=16, max_len=128):
    """Extract CLS embeddings from transformer"""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(DEVICE)
    model.eval()

    all_embeddings = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc=f"Embedding {model_name}"):
            batch_texts = texts[i:i+batch_size]
            enc = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=max_len,
                return_tensors="pt"
            ).to(DEVICE)

            outputs = model(**enc)
            # Take [CLS] token (first hidden state)
            embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeddings.append(embeddings)

    return np.vstack(all_embeddings)

# ---------------------------
# Cross-validation + SVM
# ---------------------------
def run_cv(X, y, model_name):
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)

    # Hyperparameter grid for SVM-RBF
    param_grid = {
        "C": [0.1, 1, 10],
        "gamma": ["scale", "auto", 0.01, 0.001, 0.005],
        "kernel": ["rbf"],
    }

    all_results = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"\n=== {model_name} | Fold {fold}/{FOLDS} ===")
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = np.array(y)[train_idx], np.array(y)[val_idx]

        svm = SVC()
        grid = GridSearchCV(svm, param_grid, scoring="f1_macro", cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_val)

        acc = accuracy_score(y_val, y_pred) * 100
        prec = precision_score(y_val, y_pred, average="macro") * 100
        rec = recall_score(y_val, y_pred, average="macro") * 100
        f1 = f1_score(y_val, y_pred, average="macro") * 100

        print(f"Best Params: {grid.best_params_}")
        print(f"Fold {fold} -> Acc: {acc:.2f} | Prec: {prec:.2f} | Rec: {rec:.2f} | F1: {f1:.2f}")

        all_results.append({
            "fold": fold,
            "acc": acc, "prec": prec, "rec": rec, "f1": f1,
            "best_params": grid.best_params_
        })

    # Aggregate results
    df_results = pd.DataFrame(all_results)
    mean_results = df_results.mean(numeric_only=True).to_dict()

    return mean_results, all_results

# ---------------------------
# Run pipeline
# ---------------------------
# ---------------------------
# Run pipeline
# ---------------------------
final_summary = {}

for model_name, model_ckpt in transformer_models.items():
    print("\n" + "="*20)
    print(f" Running {model_name} ({model_ckpt})")
    print("="*20)

    embeddings = get_embeddings(model_ckpt, texts)
    mean_results, all_folds = run_cv(embeddings, labels, model_name)

    # Save results
    final_summary[model_name] = {
        "accuracy": mean_results["acc"],
        "precision": mean_results["prec"],
        "recall": mean_results["rec"],
        "f1": mean_results["f1"],
    }

    # --- Print final results block ---
    print(f"\n>>> {model_name} Final CV Results ({FOLDS} folds)")
    print(f"Accuracy: {mean_results['acc']:.2f}")
    print(f"Precision: {mean_results['prec']:.2f}")
    print(f"Recall: {mean_results['rec']:.2f}")
    print(f"F1: {mean_results['f1']:.2f}")
    print("="*60)



=== Loading Dataset ===
Train File: train_subtask1.csv -> 2925 samples, 6 columns
Dev File  : dev_subtask1.csv -> 323 samples, 6 columns
Test File : test_subtask1_text.csv -> 311 samples, 2 columns


========== BART Training (10-Fold CV) ==========
Model Spec: 12 layers, 1024 hidden, 16 heads
Best Params: {'C': 1.0, 'gamma': 0.01, 'kernel': 'rbf'}

========== BART | Fold 1/10 ==========
Epoch   1/100 - acc: 69.00% - prec: 70.17% - rec: 69.78% - f1: 70.03%
Epoch   2/100 - acc: 69.05% - prec: 70.38% - rec: 70.05% - f1: 70.22%
Epoch   3/100 - acc: 69.45% - prec: 70.30% - rec: 70.48% - f1: 70.74%
Epoch   4/100 - acc: 69.40% - prec: 70.90% - rec: 70.84% - f1: 70.79%
Epoch   5/100 - acc: 69.00% - prec: 71.10% - rec: 71.10% - f1: 70.99%
Epoch   6/100 - acc: 69.00% - prec: 71.46% - rec: 70.86% - f1: 71.39%
Epoch   7/100 - acc: 69.01% - prec: 71.51% - rec: 71.55% - f1: 71.78%
Epoch   8/100 - acc: 69.00% - prec: 72.22% - rec: 71.80% - f1: 71.76%
Epoch   9/100 - acc: 69.02% - prec: 72.16% - rec: 